<a href="https://colab.research.google.com/github/iamhero2709/Learning-Projects-Notebook/blob/main/tensorflow_withdl/LSTM_for_Language_Modeling_tensorflow6_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RECURRENT NETWORKS and LSTM IN DEEP LEARNING
---

In [1]:
import tensorflow as tf
import numpy as np


In [2]:
!mkdir data
!wget -q -O data/ptb.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/ptb.zip
!unzip -o data/ptb.zip -d data
!cp data/ptb/reader.py .

Archive:  data/ptb.zip
   creating: data/ptb/
  inflating: data/ptb/reader.py      


In [3]:
import reader

Building The LSTM model for Language Moedling
---

In [4]:
# Downloading and the extracting the simple example dataset
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar xzf simple-examples.tgz -C data/

--2025-07-16 08:57:15--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.fit.vut.cz/~imikolov/rnnlm/simple-examples.tgz [following]
--2025-07-16 08:57:15--  https://www.fit.vut.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vut.cz (www.fit.vut.cz)... 147.229.9.65, 2001:67c:1220:809::93e5:941
Connecting to www.fit.vut.cz (www.fit.vut.cz)|147.229.9.65|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.fit.vut.cz/person/imikolov/public/rnnlm/simple-examples.tgz [following]
--2025-07-16 08:57:17--  https://www.fit.vut.cz/person/imikolov/public/rnnlm/simple-examples.tgz
Reusing existing connection to www.fit.vut.cz:443.
HTTP request sent, awaiting response... 200 OK


In [8]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size= 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size= 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

In [9]:
# reads the data and seperated it into training datat m validation datat and testing data
raw_data=reader.ptb_raw_datat(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [10]:
len(train_data)

929589

In [12]:
len(valid_data)

73760

In [13]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line


print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


In [15]:
#Lets just read one mini-batch now and feed our network:


In [16]:
itera=reader.ptb_iterator(train_data,batch_size,num_steps)
first_touple = itera.__next__()
_input_data = first_touple[0]
_targets = first_touple[1]

In [17]:
_input_data.shape

(30, 20)

In [18]:
_targets.shape

(30, 20)

In [19]:
#Lets look at 3 sentences of our input x:

In [20]:
_input_data[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]],
      dtype=int32)

In [23]:
print(id_to_word(_input_data[0,:]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim']


EMBEDDINGS
---

In [25]:
embedding_layer = tf.keras.layers.Embedding(vocab_size, embeding_vector_size,trainable=True,name="embedding_vocab")

In [26]:
# Define where to get the data for our embeddings from
inputs = embedding_layer(_input_data)
inputs

<tf.Tensor: shape=(30, 20, 200), dtype=float32, numpy=
array([[[-0.01436456,  0.0440993 , -0.02107005, ...,  0.01413027,
         -0.01145139, -0.03084683],
        [ 0.04215157, -0.02009675, -0.02407203, ..., -0.00023391,
          0.04727561, -0.01329543],
        [ 0.00682461, -0.02996184,  0.00371871, ...,  0.03537413,
         -0.03849957, -0.00537307],
        ...,
        [ 0.04776729, -0.01894908, -0.00567423, ...,  0.02156867,
         -0.03187786,  0.01174675],
        [-0.0490693 ,  0.0273697 ,  0.00766647, ...,  0.02603543,
          0.01303884,  0.00227611],
        [ 0.03717044, -0.02521272,  0.04218866, ...,  0.02438002,
         -0.0119619 , -0.02224101]],

       [[-0.0186911 , -0.02416588, -0.02485015, ...,  0.02687794,
         -0.03799218, -0.03854796],
        [ 0.01473199, -0.02374755,  0.02031913, ...,  0.03837327,
          0.03252325,  0.04618504],
        [-0.02834617,  0.00438561, -0.00953913, ..., -0.00511362,
          0.0188951 , -0.02685125],
        ...,

<h3>Constructing Recurrent Neural Networks</h3>


In [27]:
lstm_cell_l1 = tf.keras.layers.LSTMCell(hidden_size_l1)
lstm_cell_l2 = tf.keras.layers.LSTMCell(hidden_size_l2)

In [28]:
stacked_lstm = tf.keras.layers.StackedRNNCells([lstm_cell_l1, lstm_cell_l2])

In [29]:
layer  =  tf.keras.layers.RNN(stacked_lstm,[batch_size, num_steps],return_state=False,stateful=True,trainable=True)

In [30]:
init_state = tf.Variable(tf.zeros([batch_size,embeding_vector_size]),trainable=False)

In [31]:
layer.inital_state = init_state

In [32]:
layer.inital_state

<tf.Variable 'Variable:0' shape=(30, 200) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [33]:
outputs = layer(inputs)

In [34]:
outputs

<tf.Tensor: shape=(30, 20, 128), dtype=float32, numpy=
array([[[-1.6887874e-03, -2.2335418e-05, -9.2224707e-04, ...,
         -3.0881836e-07,  1.3239472e-03,  4.9628405e-04],
        [-1.9791326e-03,  2.6744499e-04, -1.4664220e-03, ...,
         -2.3299240e-04,  2.8962109e-03,  1.2438344e-03],
        [-4.8868014e-03,  1.2401775e-04, -2.6096166e-03, ...,
          2.6333935e-04,  4.0177279e-03,  3.3195566e-03],
        ...,
        [-1.2695015e-04,  4.7474657e-03,  1.6818448e-03, ...,
          9.6982246e-04, -1.7865861e-03,  1.1568234e-03],
        [ 2.1898847e-03,  4.0006372e-03,  3.9001126e-03, ...,
         -1.8554339e-03, -3.1012679e-03,  1.5256708e-03],
        [ 3.9156834e-03,  2.2908205e-03,  3.8792619e-03, ...,
         -3.2028779e-03, -3.0979966e-03,  1.5841797e-03]],

       [[-1.5331253e-03, -7.0395449e-04,  1.2044576e-05, ...,
          3.3090562e-06,  4.6755350e-04, -1.2553754e-03],
        [-2.4340677e-03, -1.7228093e-03, -5.1922735e-04, ...,
          4.5084156e-04,  6.

<h2>Dense layer</h2>
We now create densely-connected neural network layer that would reshape the outputs tensor from  [30 x 20 x 128] to [30 x 20 x 10000].


In [35]:
dense = tf.keras.layers.Dense(vocab_size)

In [36]:
logits_outputs  = dense(outputs)

In [37]:
print("shape of the output from dense layer: ", logits_outputs.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from dense layer:  (30, 20, 10000)


<h2>Activation layer</h2>

A softmax activation layers is also then applied to derive the probability of the output being in any of the multiclass(10000 in this case) possibilities.


In [39]:
activation = tf.keras.layers.Activation('softmax')

In [40]:
output_words_prob = activation(logits_outputs)

In [41]:
print("shape of the output from the activation layer: ", output_words_prob.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from the activation layer:  (30, 20, 10000)


In [42]:
print("The probability of observing words in t=0 to t=20", output_words_prob[0,0:num_steps])

The probability of observing words in t=0 to t=20 tf.Tensor(
[[9.99818440e-05 1.00003665e-04 1.00012316e-04 ... 9.99896438e-05
  1.00009645e-04 9.99818658e-05]
 [9.99781550e-05 9.99867480e-05 1.00017118e-04 ... 9.99580807e-05
  1.00000339e-04 9.99592303e-05]
 [1.00004509e-04 9.99633921e-05 1.00000783e-04 ... 9.99492331e-05
  9.99800395e-05 9.99444383e-05]
 ...
 [9.99958647e-05 1.00013021e-04 1.00026082e-04 ... 1.00032652e-04
  9.99937693e-05 9.98087853e-05]
 [9.99842814e-05 1.00015575e-04 1.00016900e-04 ... 1.00066092e-04
  1.00006720e-04 9.97996030e-05]
 [9.99673139e-05 1.00008918e-04 1.00030848e-04 ... 1.00091493e-04
  9.99956101e-05 9.97943280e-05]], shape=(20, 10000), dtype=float32)


<h3>Prediction</h3>
What is the word correspond to the probability output? Lets use the maximum probability:


In [43]:
np.argmax(output_words_prob[0,0:num_steps], axis=1)

array([8244, 5314, 5258, 5258, 5258, 2009, 2009, 2009, 2009, 2009, 2391,
       2391, 8010, 1848, 9846, 1317, 1317, 1317, 1317, 8012])

In [44]:
_targets[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

<h4>Objective function</h4>

How similar the predicted words are to the target words?

Now we have to define our objective function, to calculate the similarity of predicted values to ground truth, and then, penalize the model with the error. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}\_i}$$

This function is already implemented and available in TensorFlow through _tf.keras.losses.sparse_categorical_crossentropy_. It calculates the categorical cross-entropy loss for <b>logits</b> and the <b>target</b> sequence.  

The arguments of this function are:  

<ul>
    <li>logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].</li>  
    <li>targets: List of 1D batch-sized int32 Tensors of the same length as logits.</li>   
</ul>


In [45]:
def crossentropy(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

In [46]:
loss  = crossentropy(_targets, output_words_prob)

In [47]:
loss[0,:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([9.210186, 9.210302, 9.210356, 9.209693, 9.210658, 9.21028 ,
       9.210519, 9.211017, 9.209904, 9.211192], dtype=float32)>

In [48]:
cost = tf.reduce_sum(loss / batch_size)
cost

<tf.Tensor: shape=(), dtype=float32, numpy=184.207763671875>

<h3>Training</h3>

To do training for our network, we have to take the following steps:

<ol>
    <li>Define the optimizer.</li>
    <li>Assemble layers to build model.</li>
    <li>Calculate the gradients based on the loss function.</li>
    <li>Apply the optimizer to the variables/gradients tuple.</li>
</ol>





<h4>1. Define Optimizer</h4>




In [51]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=max_grad_norm)

<h4>2. Assemble layers to build model.</h4>


In [52]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(layer)
model.add(dense)
model.add(activation)
model.compile(loss=crossentropy, optimizer=optimizer)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_vocab (Embedding)     │ (30, 20, 200)          │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (RNN)                       │ (30, 20, 128)          │       665,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (30, 20, 10000)        │     1,290,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,955,088 (15.09 MB)

 Trainable params: 3,955,088 (15.09 MB)

 Non-trainable params: 0 (0.00 B)

<h4>2. Trainable Variables</h4>


Defining a variable, if you passed <i>trainable=True</i>, the variable constructor automatically adds new variables to the graph collection <b>GraphKeys.TRAINABLE_VARIABLES</b>. Now, using <i>tf.trainable_variables()</i> you can get all variables created with <b>trainable=True</b>.


In [53]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = model.trainable_variables

Note: we can find the name and scope of all variables:


In [54]:
[v.name for v in tvars]

['embeddings',
 'kernel',
 'recurrent_kernel',
 'bias',
 'kernel',
 'recurrent_kernel',
 'bias',
 'kernel',
 'bias']

<h4>3. Calculate the gradients based on the loss function</h4>
**Gradient**: The gradient of a function is the slope of its derivative (line), or in other words, the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by the <b>derivative</b> operation.
First lets recall the gradient function using an toy example:
$$ z = \left(2x^2 + 3xy\right)$$


In [55]:
x = tf.constant(1.0)
y =  tf.constant(2.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    g.watch(y)
    func_test = 2 * x * x + 3 * x * y

The <b>tf.gradients()</b> function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. <b>tf.gradients(func, xs)</b> constructs symbolic partial derivatives of sum of <b>func</b> w.r.t. <i>x</i> in <b>xs</b>.

Now, lets look at the derivitive w.r.t. <b>var_x</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 4x + 3y $$


In [56]:
var_grad = g.gradient(func_test, x) # Will compute to 10.0
print(var_grad)

tf.Tensor(10.0, shape=(), dtype=float32)


the derivative w.r.t. <b>var_y</b>:
$$ \frac{\partial \:}{\partial \:y}\left(2x^2 + 3xy\right) = 3x $$


In [57]:
var_grad = g.gradient(func_test, y) # Will compute to 3.0
print(var_grad)

tf.Tensor(3.0, shape=(), dtype=float32)


Now, we can look at gradients w.r.t all variables:


In [58]:
with tf.GradientTape() as tape:
    # Forward pass.
    output_words_prob = model(_input_data)
    # Loss value for this batch.
    loss  = crossentropy(_targets, output_words_prob)
    cost = tf.reduce_sum(loss,axis=0) / batch_size

In [59]:
# Get gradients of loss wrt the trainable variables.
grad_t_list = tape.gradient(cost, tvars)

In [60]:
print(grad_t_list)

[<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x78289d2346d0>, <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
array([[-6.6341090e-07, -1.7943911e-06,  1.2738228e-06, ...,
         1.4700987e-06, -1.9430928e-09,  1.8701965e-07],
       [-1.1954504e-07, -8.6421505e-07,  2.6428540e-07, ...,
         2.5293861e-07,  1.8830329e-07, -6.4357785e-08],
       [-9.1166379e-07, -1.0903372e-06, -1.3817676e-06, ...,
         8.6296490e-07,  2.2890774e-07, -2.4464435e-07],
       ...,
       [ 9.2097594e-08, -2.2139028e-07,  5.9687051e-07, ...,
        -4.1208261e-07, -8.1399963e-07,  3.2546933e-07],
       [ 5.7409784e-07,  3.2205017e-07,  8.2884753e-07, ...,
         3.1615627e-07, -1.4287221e-07,  5.7324735e-07],
       [ 2.3974297e-07,  1.8661331e-07, -1.1566767e-06, ...,
         3.6195331e-08, -2.9375056e-07, -1.0501839e-07]], dtype=float32)>, <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
array([[ 2.48230947e-07,  6.90864255e-08, -3.91676167e-08, ...,
      



now, we have a list of tensors, t-list. We can use it to find clipped tensors. <b>clip_by_global_norm</b> clips values of multiple tensors by the ratio of the sum of their norms.

<b>clip_by_global_norm</b> get <i>t-list</i> as input and returns 2 things:

<ul>
    <li>a list of clipped tensors, so called <i>list_clipped</i></li>
    <li>the global norm (global_norm) of all tensors in t_list</li>
</ul>


In [61]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

 <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
 array([[-6.6341090e-07, -1.7943911e-06,  1.2738228e-06, ...,
          1.4700987e-06, -1.9430928e-09,  1.8701965e-07],
        [-1.1954504e-07, -8.6421505e-07,  2.6428540e-07, ...,
          2.5293861e-07,  1.8830329e-07, -6.4357785e-08],
        [-9.1166379e-07, -1.0903372e-06, -1.3817676e-06, ...,
          8.6296490e-07,  2.2890774e-07, -2.4464435e-07],
        ...,
        [ 9.2097594e-08, -2.2139028e-07,  5.9687051e-07, ...,
         -4.1208261e-07, -8.1399963e-07,  3.2546933e-07],
        [ 5.7409784e-07,  3.2205017e-07,  8.2884753e-07, ...,
          3.1615627e-07, -1.4287221e-07,  5.7324735e-07],
        [ 2.3974297e-07,  1.8661331e-07, -1.1566767e-06, ...,
          3.6195331e-08, -2.9375056e-07, -1.0501839e-07]], dtype=float32)>,
 <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
 array([[ 2.48230947e-07,  6.90864255e-08, -3.91676167e-08, ...,
         -5.76186324e-07,  3.17301208e-08, -3.51071314e-08],
        [ 1.398

<h4> 4.Apply the optimizer to the variables/gradients tuple. </h4>


In [62]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [76]:
!wget -O lyrics.csv https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2025-07-16 09:33:53--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘lyrics.csv’

lyrics.csv          100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-16 09:33:53 (17.3 MB/s) - ‘lyrics.csv’ saved [1115394/1115394]

